# `Selenium Webscraping Indeed Job Postings - July 2023`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

# `Purpose & Outcome:`

# `What is Selenium and how is it used?`

+ When you need to do unit testing, automation or assistance when webscraping this is a tool to aid you.
    + Great for clicking buttons
    + drop-down menus
    + acting/emulating human interactions on a webpage

In [1]:
# Install if you have never used these: unblock the lines below to install if needed

# !pip install webdriver-manager
# !pip3 install lxml
# !pip3 install selenium
# !pip3 install webdriver_manager
# !pip install --upgrade pip
# !pip install -U selenium

In [2]:
# --------- import necessary modules -------

# For webscraping
from bs4 import BeautifulSoup

# Parsing and creating xml data
from lxml import etree as et

# Store data as a csv file written out
from csv import writer

# In general to use with timing our function calls to Indeed
import time

# Assist with creating incremental timing for our scraping to seem more human
from time import sleep

# Dataframe stuff
import pandas as pd

# Random integer for more realistic timing for clicks, buttons and searches during scraping
from random import randint

In [3]:
import selenium

# Check version I am running
selenium.__version__

'4.10.0'

In [4]:
# selenium 4:

from selenium import webdriver

from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager

In [5]:
# Allows searchs similar to beautiful soup: find_all
from selenium.webdriver.common.by import By

# Try to establish wait times for the page to load
from selenium.webdriver.support.ui import WebDriverWait

# Wait for specific condition based on defined task: web elements, boolean are examples
from selenium.webdriver.support import expected_conditions as EC

# Used for keyboard movements, up/down, left/right,delete, etc
from selenium.webdriver.common.keys import Keys

# Locate elements on page and throw error if they do not exist
from selenium.common.exceptions import NoSuchElementException

# `Consider Headless Browser: speed up & use less resources:`

There are some condiserations though:

+ Some browsers create issues
+ debugging can be tricky
+ you may have limited plugin usage or support
+ you are not able to see visually how the website or application are working 

# `from selenium.webdriver.common.by import By`

Think of this as being similar to using `Beautiful Soup and find_all`
+ when used it allows you to find something within an HTML document, if it fails you raise the exception: `NoSuchElementException`
+ **`Becareful when using BY`** because if this is not a static page then any attrubutes you are searching can become an error in the future when it fails.
    + For example if you are searching by `Class` this can create issues later vs using
        + This is because it is a `CSS` selector and can change overtime since it is an attribute
    + `ID` which may make your code more robust! This CAN be a unique identifier that may help you instead

# `NoSuchElementException`

This is useful to locate elements within a page while loading and try to handle exceptions.
+ During `AJAX` calls you may have issues if the application was build using `React, VUE, Angular` and require different use cases to make the above checks. [article to explain](https://reflect.run/articles/everything-you-need-to-know-about-nosuchelementexception-in-selenium/) and you can consider polling.

`--------------------------------`

# `Other Common Errors:`

+ **`InvalidSelectorException`**

+ **`ElementNotInteractableException`**

+ **`TimeoutException`**

In [6]:
option= webdriver.ChromeOptions()

# Going undercover:
option.add_argument("--incognito")


# # Consider this if the application works and you know how it works for speed ups and rendering!

# option.add_argument('--headless=chrome')


In [422]:
# Define job and location search keywords
job_search_keyword = ['Data+Scientist', 'Business+Analyst', 'Data+Engineer', 
                      'Python+Developer', 'Full+Stack+Developer', 'Machine+Learning+Engineer']

# Define Locations of Interest
location_search_keyword = ['New+York', 'California', 'Washington']

# Define base and pagination URL's
base_url = 'https://www.indeed.com'

# Finding location, position, radius=35 miles, sort by date and starting page
paginaton_url = 'https://www.indeed.com/jobs?q={}&l={}&radius=35&filter=0&sort=date&start={}'

print(paginaton_url)

https://www.indeed.com/jobs?q={}&l={}&radius=35&filter=0&sort=date&start={}


# Things to consider:

+ Wait for page to load before we start running tasks
+ make sure what we are looking for is actually there
    + It can be absent
    + hidden in DOM, iframe or similar
+ timing our calls to remain more like an average user
+ Exception handling

# `Let's Look at what is going on below:`

There are some concerns and things for you to consider:

1.) Below is the MAX number of jobs you will find for a posting of interest!

2.) This is not an accurate depiction because you can have way less than this depending on results
    
    + An issue arises due to duplicated listings

3.) Pagination is difficult to do and when to stop the search results

4.) you have a filter option (&filter=0, &filter=1), filter =1 shows non-duplicates which reduces results but you need to figure out how to do pagination!

In [428]:
job_='Data+Engineer'
location='Washington'
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),
                         options=option)

driver.get(paginaton_url.format(job_,location,0))
sleep(randint(2, 6))
p=driver.find_element(By.CLASS_NAME,'jobsearch-JobCountAndSortPane-jobCount').text

u=math.ceil(int(p.split(' ')[0])/15)
driver.quit()
u

# for i in range(0,11*10+10,10):
#     print(i)

12

In [429]:
# pagination: PRACTICE

job_='Data+Engineer'
location='Washington'
# page=[0,10,20]


job_lst=[]


job_description_list = []
salary_list=[]


driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),
                         options=option)
sleep(randint(2, 6))

# driver.get("https://www.indeed.com/q-USA-jobs.html")

# for i in page:
for i in range(0,u):
    driver.get(paginaton_url.format(job_,location,i*10))
    sleep(randint(2, 4))

    job_page = driver.find_element(By.ID,"mosaic-jobResults")
    jobs = job_page.find_elements(By.CLASS_NAME,"job_seen_beacon") # return a list

    for jj in jobs:
        job_title = jj.find_element(By.CLASS_NAME,"jobTitle")
#         print(job_title.text)
        
# Href's to get full job description (need to re-terate to get full info)
# Reference ID for each job used by indeed         
# Finding the company name        
# Location
# Posting date
# Job description

        job_lst.append([job_title.text,
        job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("href"),
        job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("id"),      
        jj.find_element(By.CLASS_NAME,"companyName").text,       
        jj.find_element(By.CLASS_NAME,"companyLocation").text,
        jj.find_element(By.CLASS_NAME,"date").text,
        job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("href")])
    
    
        # Job Description Links for further parsing and scraping text
#         job_description_list.append(job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("href"))
        

        try: 
            salary_list.append(jj.find_element(By.CLASS_NAME,"metadata salary-snippet-container"))

        except NoSuchElementException: 
            try: 
                salary_list.append(jj.find_element(By.CLASS_NAME,"estimated-salary").text)
                
            except NoSuchElementException:
                salary_list.append(None)

                
                
        # Click the job element to get the description
        job_title.click()
        
        # Help to load page so we can find and extract data
        sleep(randint(3, 5))

        try: 
            job_description_list.append(driver.find_element(By.ID,"jobDescriptionText").text)
            
        except: 
            
            job_description_list.append(None)

            
# alternate way to grab the info for job description to make it faster:


In [128]:
# Get links and take description:

# Salary if available and use exceptions

# remote or not! parse with regex and use a separate column for df

#setup for number of jobs and parse by that as iterator

In [491]:
# convert dates using date time
# job_lst[-2]

# job_description_list

['Senior Software Engineer, Big Data Processing',
 'https://www.indeed.com/rc/clk?jk=9a67e5a624e1f04d&fccid=642127b94898142b&vjs=3',
 'job_9a67e5a624e1f04d',
 'SpaceCurve',
 'Seattle, WA 98104',
 'Posted\nPosted 30+ days ago',
 'https://www.indeed.com/rc/clk?jk=9a67e5a624e1f04d&fccid=642127b94898142b&vjs=3']

In [496]:
paginaton_url_ = 'https://www.indeed.com/jobs?q={}&l={}&sort=date&start={}'

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),
                         options=option)
p_=[]
salary_list_=[]
for i in range(0,3):
    driver.get(paginaton_url_.format(job_,location,i*10))
    sleep(randint(2, 3))
    
    job_page = driver.find_element(By.ID,"mosaic-jobResults")
    jobs = job_page.find_elements(By.CLASS_NAME,"job_seen_beacon") # return a list
    
    for jj in jobs:
        job_title = jj.find_element(By.CLASS_NAME,"jobTitle")
        print(job_title.text)
        p_.append(job_title.text)
#         sleep(randint(3, 5))
        try:
            salary_list_.append(jj.find_element(By.CLASS_NAME,"salary-snippet-container").text)
            print(jj.find_element(By.CLASS_NAME,"salary-snippet-container").text)
#         sleep(randint(1, 2))
#         try: 
#             salary_list_.append(jj.find_element(By.CLASS_NAME,"salary-snippet-container").text)
#             print(jj.find_element(By.CLASS_NAME,"salary-snippet-container").text)
        except: 
            try: 
                salary_list.append(jj.find_element(By.CLASS_NAME,"estimated-salary").text)
                print(jj.find_element(By.CLASS_NAME,"estimated-salary").text)
            except:
                print('None')
                
driver.quit()

# //*[@id="challenge-stage"]/div/label/input

Data Engineer
None
Artica - Senior Data Applied Science Engineer (Seattle, WA) - Direct Hire [Hybrid]
$180,000 - $200,000 a year
Senior Data Engineer (US Remote)
Estimated $125K - $158K a year
Data Engineer
$140,000 - $190,000 a year
Staff Software Engineer - Data Science
$149,240 - $200,200 a year
Staff Software Engineer - Data Integration
$149,240 - $200,200 a year
Principal Software Engineer (Data), Industry Solutions Engineering
$133,600 - $256,800 a year
Data Engineer, AET Central Services Technology - Data
From $105,700 a year
Software Engineer, Data Platform
Estimated $137K - $174K a year
Customer Engineer, Data Analytics, Google Cloud
None
Software Engineer, Data Platform
None
Senior Backend Engineer - Data
$140,000 - $215,000 a year
Staff Full Stack Engineer - Data
$175,000 - $225,000 a year
Distributed Data Systems - Staff Software Engineer
$182,400 - $247,000 a year
Data Center Operations Engineer (Kirkland, WA)
$45 - $51 an hour
Senior Software Engineer - Distributed Data S

In [464]:
# salary_list_

In [130]:
# df and store data


# duplicate entries remove!

In [131]:
# plots

In [ ]:
# consider NLP


# `Here is a side note:`


+ This gives me an error because it was code from the past version:

`driver = webdriver.Chrome(ChromeDriverManager().install())`


+ `When using ingonito browser:` your browsing tabs will pull different data than a normal window. Understand this when doing your troubleshooting and debugging. If you have a window open to find your tags but parse in a different type of window the results will not line up.

+ Also, when you are grabbing `job descriptions` for example you will need to time it so the page will read the data after it is loaded. If you immediately try to grab data you may not get everything!
    + Option 1: use the clickable tab from the `job title` then scrape directly
    + Option 2: consider saving the `HREF's` and then doing a separate parsing in a different function. This I think may be faster. But, check for yourself.
    
+ To speed things up consider `headless browser` but, understand the debugging becomes an issue!

+ **If you parse a good amount of pages** you will encounter a checkbox that needs to be clicked to show you are not a robot. This occurs to me usually after 15-30 pages of scraping which is not a lot. (I need to figure this out)
    + Option 1: try to see if you can pull the information for this button to scrape it directly and click
    + Option 2: reset and tinker with the settings of timing out, sleep settings and maybe error handling

**`Big Concern: Pagination`**
When you need to go from page to page sequentially this is not straight forward. Practice and a lot of reading will aid you. I am not savvy just yet.
+ Clickable buttons and learning how to use them and WHEN TO STOP iterating are NOT trivial tasks
+ Hacking your way through, such I did for this example but, there is a glaring issue with duplicate entries.
+ Finding hidden elements and figuring out how to extract them.

# First let's try to find the number of jobs for a given posting


In [ ]:
# driver.find_element(By.ID,"mosaic-jobResults").text
# ("//div[@class='jobsearch-SerpJobCard unifiedRow row result clickcard']")
# .text
# find_element_by_xpath("//div[@class='jobsearch-SerpJobCard unifiedRow row result clickcard']").text


# 1.) iterate over jobs, locations
# 2.) find the elements from outer to inner
# 3.) extract everything then go to next page
# 4.) medium article for each pagination
# 5.) error handling & timing, clicking

In [ ]:
# Consider a few options:

# 1.) Try to use incognito -----------(done)
# 2.) Maybe I should use random int for sleep ----(good idea)
# 3.) What to do when I have the human click button for pop up
# 4.) verify if the "+" symbols are needed look at formatting ----------(done)
# 5.) check if the formatting to parse is the same or not for div tags, etc
# 6.) do I need to use headless browser? (Possible issues! investigate)
# 7.) locating elements using the BY package, similar to beautiful soup find_all
# 8.) errors with NoSuchElementException ----------------(done)
# 9.) try to identify code that doesn't change over time
# 10.) Xpath to find buttons to go page by page and contain arrows forward/backward with try.except

# Notes for this project:

+ Filling in forms:
+ click buttons
+ possible human detection stuff

**`Xpath vs CSS selectors for retreiving data`**

+ `Xpath:` bidirectional (can go from parent to child and reverse) traversal
    + slower retrevial speed
    + text functions supported
    + pay attention to relative '//' and absolute path '/' notations
    + Think of a tree like structure to breakdown
+ `CSS:` directional (parent to child only)

`------------------------`

**`Xpath`**
+ *`Xpath`* stands for `XML Path` which is a query language used to find the path of an element in XML documents
+ Essentially you are navigating a `DOM` 
+ More flexible than using `CSS`
    + If you don't know the name of an element you can use `contains` as your key word which is great!
 
**`CSS`**
+ Most often the HTML will be styled in a cascading format and identifying elements will come from the `Class` they fall within
+ They are used to select various elements within a `DOM`
    + **`Simple selectors:`** such as finding a `Class` or `ID`
    + **`Attribute selectors:`** 
    + **`Pseudo selectors:`** such as hover boxes or check boxes as examples

In [ ]:
url="https://www.indeed.com/q-USA-jobs.html?vjk=823cd7ee3c203ac3"

In [ ]:
# function to get DOM from given URL
def get_dom(url):
    driver.get(url)
    page_content = driver.page_source
    product_soup = BeautifulSoup(page_content, 'html.parser')
    dom = et.HTML(str(product_soup))
    return dom

In [ ]:
# for i in get_dom(url):
#     print(et.tostring(i))

In [ ]:
# functions to extract job link
# def get_job_link(job):
#     practice_=[]
#     try:
#         job_link = job.xpath('./descendant::h2/a/@href')[0]
#     except Exception as e:
#         job_link = 'Not available'
#     practice_.append(job_link)
# #     return job_link
#     return practice_

# for job_keyword in job_search_keyword:
#     for location_keyword in location_search_keyword:
#         all_jobs = []
#         for page_no in range(0, 10, 10):
#             url = paginaton_url.format(job_keyword, location_keyword, page_no)
#             page_dom = get_dom(url)
#             jobs = page_dom.xpath('//div[@class="job_seen_beacon"]')
#         all_jobs.append(jobs)
# #             all_jobs = all_jobs + jobs
# #     print(all_jobs)


In [ ]:
# Open a CSV file to write the job listings data
# with open('indeed_jobs1.csv', 'w', newline='', encoding='utf-8') as f:
#     theWriter = writer(f)
#     heading = ['job_link', 'job_title', 'company_name', 'company_location', 'salary', 'job_type', 'rating', 'job_description', 'searched_job', 'searched_location']
#     theWriter.writerow(heading)
#     for job_keyword in job_search_keyword:
#         for location_keyword in location_search_keyword:
#             all_jobs = []
#             for page_no in range(0, 100, 10):
#                 url = paginaton_url.format(job_keyword, location_keyword, page_no)
#                 page_dom = get_dom(url)
#                 jobs = page_dom.xpath('//div[@class="job_seen_beacon"]')
# #                 all_jobs = all_jobs +jobs
#                 print(all_jobs+jobs)
#                 all_jobs_ = all_jobs.append(jobs) #changed here and below
#                 print("yay",all_jobs_)
#             for job in all_jobs_:
#                 job_link = base_url + get_job_link(job)
#                 time.sleep(2)
#                 job_title = get_job_title(job)
#                 time.sleep(2)
#                 company_name = get_company_name(job)
#                 time.sleep(2)
#                 company_location = get_company_location(job)
#                 time.sleep(2)
#                 salary = get_salary(job)
#                 time.sleep(2)
#                 job_type = get_job_type(job)
#                 time.sleep(2)
#                 rating = get_rating(job)
#                 time.sleep(2)
#                 job_desc = get_job_desc(job)
#                 time.sleep(2)
#                 record = [job_link, job_title, company_name, company_location, salary, job_type, rating, job_desc, job_keyword, location_keyword]
#                 theWriter.writerow(record)

# # Closing the web browser
# driver.quit()

In [ ]:
import pandas as pd

# pd.read_csv('indeed_jobs1.csv')

# Like, Share & <font color=red>SUB</font>scribe

# `Citations & Help:`

# ◔̯◔

https://pypi.org/project/webdriver-manager/

https://www.blog.datahut.co/post/scrape-indeed-using-selenium-and-beautifulsoup

https://github.com/henrionantony/Dynamic-Web-Scraping-using-Python-and-Selenium/blob/master/indeed.py

https://www.specrom.com/blog/web-scraping-job-postings-on-indeed-using-python/

https://www.scrapingdog.com/blog/scrape-indeed-using-python/ (bs4 as of Feb 13, 2023)

https://selenium-python.readthedocs.io/locating-elements.html#locating-elements

https://stackoverflow.com/questions/50865088/how-to-get-string-dump-of-lxml-element

https://selenium-python.readthedocs.io/navigating.html

https://towardsdatascience.com/web-scraping-job-postings-from-indeed-com-using-selenium-5ae58d155daf (2020 version)

https://www.pycodemates.com/2022/01/Indeed-jobs-scraping-with-python-bs4-selenium-and-pandas.html

https://medium.com/forcodesake/how-to-build-a-scraping-tool-for-indeed-in-8-minutes-data-science-csv-selenium-beautifulsoup-python-95fcca4b9719 (Good Read & Adapted Code)

https://www.tutorialspoint.com/how-to-open-browser-window-in-incognito-private-mode-using-python-selenium-webdriver

https://www.selenium.dev/selenium/docs/api/py/webdriver/selenium.webdriver.common.keys.html

https://pythonbasics.org/selenium-wait-for-page-to-load/

https://www.seleniumeasy.com/selenium-tutorials/selenium-headless-browser-execution

https://www.browserstack.com/guide/expectedconditions-in-selenium

https://www.testim.io/blog/xpath-vs-css-selector-difference-choose/

https://www.w3.org/TR/REC-DOM-Level-1/introduction.html

https://github.com/diego-florez/Selenium-Web-Scraping/blob/master/indeed.py (Indeed scrape Selenium 2020) error Handling also

https://www.testim.io/blog/selenium-click-button/

https://scrapfly.io/blog/how-to-scrape-indeedcom/

https://goh.physics.ucdavis.edu/datascience/webscraping/webscraping.html

https://levelup.gitconnected.com/efficiently-scraping-multiple-pages-of-data-a-guide-to-handling-pagination-with-selenium-and-3ed93857f596

https://github.com/israel-dryer/Indeed-Job-Scraper/blob/master/indeed-job-scraper-selenium.ipynb

https://www.zenrows.com/blog/headless-browser-python#switch-to-python-selenium-headless-mode